In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import modules.logger_tool as logger
import os

os.environ['LOG_NAME'] = 'science'
os.environ['LOG_DIR'] = 'logs'
os.environ['LOG_LEVEL'] = 'INFO'

logging = logger.get_logger(os.environ['LOG_NAME'], log_level=os.environ['LOG_LEVEL'], log_path=os.environ['LOG_DIR'], log_file=os.environ['LOG_NAME'])


In [2]:
import modules.database.schemas.curriculum_neo as neo_curriculum
import modules.database.schemas.relationships.curricular_relationships as neo_relationships
import modules.database.tools.xl_planner_tools as planner
import modules.database.tools.neontology_tools as neon
import modules.database.tools.neo4j_driver_tools as driver_tools
import modules.database.tools.neo4j_session_tools as session_tools

import requests
from pydantic import ValidationError


In [3]:
db_name = 'science'

In [4]:
url = 'http://localhost:9500/database/admin/stop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = 'http://localhost:9500/database/admin/drop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

2024-06-30 01:30:04,306 INFO      : 3522179693 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAu5A="]}
2024-06-30 01:30:06,366 INFO      : 3522179693 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAvJA="]}


In [5]:
url = 'http://localhost:9500/database/admin/create-database'
params = {'db_name': db_name}
response = requests.post(url, params=params)
logging.info(response.text)

2024-06-30 01:30:08,449 INFO      : 2595822508 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAvZA="]}


In [6]:
driver = driver_tools.get_driver(database=db_name)


2024-06-30 01:30:08,515 INFO      : neo4j_driver_tools > get_driver >>> Connection successful


In [7]:
neon.init_neo4j_connection()


2024-06-30 01:30:08,555 INFO      : neontology_tools > init_neo4j_connection >>> Neontology connection initialized with host: 192.168.0.20, port: 7687, user: neo4j


In [8]:
excel_planner = planner.create_dataframes(os.getenv("EXCEL_PLANNER_FILE"))

topic_df = excel_planner['topiclookup_df']
lesson_df = excel_planner['lessonlookup_df']
statement_df = excel_planner['statementlookup_df']
keystagesyllabus_df = excel_planner['keystagesyllabuslookup_df']
yeargroupsyllabus_df = excel_planner['yeargroupsyllabuslookup_df']


In [9]:
default_topic_values = {
    'topic_assessment_type': 'Null',
    'topic_type': 'Null',
    'total_number_of_lessons_for_topic': '1',
    'topic_title': 'Null'
    }

default_topic_lesson_values = {
    'topic_lesson_title': 'Null',
    'topic_lesson_type': 'Null',
    'topic_lesson_length': '1',  # Ensure this is a string if the model expects a string
    'topic_lesson_suggested_activities': 'Null',
    'topic_lesson_skills_learned': 'Null',
    'topic_lesson_weblinks': 'Null',
}

default_learning_statement_values = {
    # Add default values for fields that might contain NaN
    'lesson_learning_statement': 'Null',
    'lesson_learning_statement_type': 'Student learning outcome'
}


In [10]:
# First, create a dictionary to store nodes by their IDs to reuse in relationships
node_library = {}

# Create KeyStageSyllabus and YearGroupSyllabus nodes with relationships
for index, ks_row in keystagesyllabus_df.iterrows():
    # Create KeyStageNode and KeyStageSyllabusNode
    key_stage_node = neo_curriculum.KeyStageNode(
        key_stage_id=str(ks_row['KeyStage']),
        key_stage_name=f"Key Stage {ks_row['KeyStage']}"
    )
    key_stage_syllabus_node = neo_curriculum.KeyStageSyllabusNode(
        ks_syllabus_id=ks_row['ID'],
        ks_syllabus_name=ks_row['Title']
    )

    # Save nodes to library
    node_library[ks_row['ID']] = key_stage_syllabus_node

    # Create or merge nodes in the database
    neon.create_or_merge_neontology_node(key_stage_node, database=db_name, operation='merge')
    neon.create_or_merge_neontology_node(key_stage_syllabus_node, database=db_name, operation='merge')

    # Create relationships between KeyStageNode and KeyStageSyllabusNode
    neon.create_or_merge_neontology_relationship(
        neo_relationships.KeyStageIncludesKeyStageSyllabus(source=key_stage_node, target=key_stage_syllabus_node), 
        database=db_name, operation='merge'
    )

    # Establish relationships with YearGroupSyllabusNodes
    related_yeargroups = yeargroupsyllabus_df[yeargroupsyllabus_df['KeyStage'] == ks_row['KeyStage']]
    for yg_index, yg_row in related_yeargroups.iterrows():        
        if str(yg_row['YearGroup']).isdigit():
            year_group_node = neo_curriculum.YearGroupNode(
                year_group_id=str(yg_row['YearGroup']), 
                year_group_name=f"Year {yg_row['YearGroup']}"
            )
            year_group_syllabus_node = neo_curriculum.YearGroupSyllabusNode(
                yr_syllabus_id=yg_row['ID'], 
                yr_syllabus_name=yg_row['Title']
            )

            # Save nodes to library
            node_library[yg_row['ID']] = year_group_syllabus_node

            # Create or merge YearGroupNode and YearGroupSyllabusNode
            neon.create_or_merge_neontology_node(year_group_node, database=db_name, operation='merge')
            neon.create_or_merge_neontology_node(year_group_syllabus_node, database=db_name, operation='merge')

            # Establish relationships
            neon.create_or_merge_neontology_relationship(
                neo_relationships.YearGroupHasYearGroupSyllabus(source=year_group_node, target=year_group_syllabus_node), 
                database=db_name, operation='merge'
            )
            neon.create_or_merge_neontology_relationship(
                neo_relationships.KeyStageSyllabusIncludesYearGroupSyllabus(source=key_stage_syllabus_node, target=year_group_syllabus_node), 
                database=db_name, operation='merge'
            )

# Process topics, lessons, and statements
for index, topic_row in topic_df.iterrows():
    yr_syllabus_node = node_library.get(topic_row['SyllabusYearID'])

    if yr_syllabus_node is not None:  # Ensure the node exists before proceeding
        # Creating topic nodes with potential NaN handling
        logging.info(f"Processing topic {topic_row['TopicID']} for year group syllabus {topic_row['SyllabusYearID']}")
        topic_node_data = {
            'topic_id': topic_row['TopicID'],
            'topic_title': topic_row.get('TopicTitle', default_topic_values['topic_title']),  # Ensure handling of NaN for title
            'total_number_of_lessons_for_topic': str(topic_row.get('TotalNumberOfLessonsForTopic', default_topic_values['total_number_of_lessons_for_topic'])),
            'topic_type': topic_row.get('TopicType', default_topic_values['topic_type']),
            'topic_assessment_type': topic_row.get('TopicAssessmentType', default_topic_values['topic_assessment_type'])
        }
        topic_node_data = planner.replace_nan_with_default(topic_node_data, default_topic_values)
        topic_node = neo_curriculum.TopicNode(**topic_node_data)
        neon.create_or_merge_neontology_node(topic_node, database=db_name, operation='merge')

        # Establish relationship between the topic node and year group syllabus node
        neon.create_or_merge_neontology_relationship(
            neo_relationships.TopicPartOfYearGroupSyllabus(source=yr_syllabus_node, target=topic_node), 
            database=db_name, operation='merge'
        )

        # Process each lesson related to the topic
        for lesson_index, lesson_row in lesson_df[lesson_df['TopicID'] == topic_row['TopicID']].iterrows():
            lesson_node_data = {
                'topic_lesson_id': lesson_row['LessonID'],
                'topic_lesson_title': lesson_row.get('LessonTitle', default_topic_lesson_values['topic_lesson_title']),
                'topic_lesson_type': lesson_row.get('LessonType', default_topic_lesson_values['topic_lesson_type']),
                'topic_lesson_length': str(lesson_row.get('SuggestedNumberOfPeriodsForLesson', default_topic_lesson_values['topic_lesson_length'])),
                'topic_lesson_suggested_activities': lesson_row.get('SuggestedActivities', default_topic_lesson_values['topic_lesson_suggested_activities']),
                'topic_lesson_skills_learned': lesson_row.get('SkillsLearned', default_topic_lesson_values['topic_lesson_skills_learned']),
                'topic_lesson_weblinks': lesson_row.get('WebLinks', default_topic_lesson_values['topic_lesson_weblinks'])
            }
            lesson_node_data = planner.replace_nan_with_default(lesson_node_data, default_topic_lesson_values)
            lesson_node = neo_curriculum.TopicLessonNode(**lesson_node_data)
            neon.create_or_merge_neontology_node(lesson_node, database=db_name, operation='merge')

            # Establish relationship between topic node and lesson node
            neon.create_or_merge_neontology_relationship(
                neo_relationships.TopicIncludesTopicLesson(source=topic_node, target=lesson_node),
                database=db_name, operation='merge'
            )

            # Process each learning statement related to the lesson
            for statement_index, statement_row in statement_df[statement_df['LessonID'] == lesson_row['LessonID']].iterrows():
                statement_node_data = {
                    'lesson_learning_statement_id': statement_row['StatementID'],
                    'lesson_learning_statement': statement_row.get('LearningOutcomeStatement', default_learning_statement_values['lesson_learning_statement']),
                    'lesson_learning_statement_type': statement_row.get('LearningStatementType', default_learning_statement_values['lesson_learning_statement_type'])
                }
                statement_node_data = planner.replace_nan_with_default(statement_node_data, default_learning_statement_values)
                statement_node = neo_curriculum.LearningStatementNode(**statement_node_data)
                neon.create_or_merge_neontology_node(statement_node, database=db_name, operation='merge')

                # Establish relationship between lesson node and learning statement node
                neon.create_or_merge_neontology_relationship(
                    neo_relationships.LessonIncludesLearningStatement(source=lesson_node, target=statement_node),
                    database=db_name, operation='merge'
                )


2024-06-30 01:30:28,301 INFO      : 1810136772 > <module> >>> Processing topic 7B1 for year group syllabus Y7.Science
2024-06-30 01:30:28,384 INFO      : 1810136772 > <module> >>> Processing lesson 7B1.1 for topic 7B1
2024-06-30 01:30:28,469 INFO      : 1810136772 > <module> >>> Processing learning statement 7B1.1.1 for lesson 7B1.1
2024-06-30 01:30:28,548 INFO      : 1810136772 > <module> >>> Processing learning statement 7B1.1.2 for lesson 7B1.1
2024-06-30 01:30:28,594 INFO      : 1810136772 > <module> >>> Processing learning statement 7B1.1.3 for lesson 7B1.1
2024-06-30 01:30:28,634 INFO      : 1810136772 > <module> >>> Processing lesson 7B1.2 for topic 7B1
2024-06-30 01:30:28,683 INFO      : 1810136772 > <module> >>> Processing learning statement 7B1.2.1 for lesson 7B1.2
2024-06-30 01:30:28,721 INFO      : 1810136772 > <module> >>> Processing learning statement 7B1.2.2 for lesson 7B1.2
2024-06-30 01:30:28,764 INFO      : 1810136772 > <module> >>> Processing learning statement 7B1.2